# GERIT MATCH

In [77]:
# Aktuelles Arbeitsverzeichnis ermitteln
import os
os.getcwd()
os.chdir("c:/Users/Hueck/OneDrive/Dokumente/GitHub/gerit_matching")

Lade Daten

In [78]:
import pandas as pd

db_hhu = pd.read_csv("data\hhu_db_raw.csv")
#db_hhu = db_hhu.sample(n=200, random_state=42)  # random_state für Reproduzierbarkeit

hhu_gerit = pd.read_excel("data/hhu_gerit.xlsx")

Orgas manuell matchen

In [79]:
# Dictionary für schnellen Lookup aus hhu_gerot erstellen
mapping_dict = dict(zip(hhu_gerit["Einrichtung"], hhu_gerit["Einrichtung"]))

# Exakte Matches zuordnen, nicht gefundene bleiben NaN
db_hhu["matched_organisation"] = db_hhu["organisation_mehrere"].map(mapping_dict)


### LLM laden



In [80]:
from dotenv import load_dotenv

# Lade Umgebungsvariablen aus der .env Datei
load_dotenv()

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [81]:
import openai
import pandas as pd
from typing import List
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain_core.runnables import chain
from langchain.agents.agent_toolkits import create_retriever_tool

# Funktion zur Erstellung von Embeddings mit dem neuen Modell
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Erstelle InMemoryVectorStore für GERIT-Organisationen
vector_store = InMemoryVectorStore(embeddings)

# Alle Organisationen aus GERIT zu den Vektoren hinzufügen
docs = [Document(page_content=einrichtung) for einrichtung in hhu_gerit["Einrichtung"].tolist()]
vector_store.add_documents(docs)

# Definiere den Retriever als Chain mit Scores
@chain
def retriever(query: str) -> List[Document]:
    results = vector_store.similarity_search_with_score(query, k=5)
    docs, scores = zip(*results) if results else ([], [])
    
    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score  # Füge den Score als Metadaten hinzu

    return list(docs)

# Beschreibung für den Retriever
description = (
    "Sucht nach gültigen Eigennamen basierend auf einer ungefähren Eingabe und gibt die Ähnlichkeitsscores zurück. "
    "Falls keine passende Entsprechung gefunden wird, wird 'NA' zurückgegeben."
)

# Retriever-Tool erstellen
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns_with_score",
    description=description,
)


In [82]:
query = "Chemie"
results = retriever.invoke(query)

# Ergebnisse anzeigen
for doc in results:
    print(f"Text: {doc.page_content}, Score: {doc.metadata['score']}")


Text: Fach Chemie, Score: 0.7509366400708087
Text: Fach Physik, Score: 0.5848930936229463
Text: Institut für Physikalische Chemie, Score: 0.5569949476123793
Text: Fach Pharmazie, Score: 0.5564846229056805
Text: Lehrstuhl für Organische Chemie, Score: 0.5424612309893898


In [68]:

# Funktion zum Abgleichen von Organisationen
def match_organisation(query, mapping_dict, retriever):
    if not isinstance(query, str):
        return None  # Falls query kein String ist
    
    orgs = [org.strip() for org in query.split(';')]  # Aufteilen und Leerzeichen entfernen
    matched_orgs = []

    for org in orgs:
        # 1. Prüfen, ob eine exakte Übereinstimmung existiert
        if org in mapping_dict:
            matched_orgs.append(mapping_dict[org])
        else:
            # 2. Falls keine exakte Übereinstimmung, unscharfe Suche über den Retriever
            results = retriever.invoke(org)  # invoke nutzen, da retriever eine Chain ist

            # Sicherstellen, dass ein Ergebnis existiert und den Score-Threshold erfüllt
            if results and results[0].metadata.get("score", 0) >= 0.7:
                matched_orgs.append(results[0].page_content)

    return "; ".join(matched_orgs) if matched_orgs else None  # Falls Matches vorhanden, mit "; " verbinden


In [ ]:

# Dictionary für exakte Übereinstimmungen erstellen
gerit_orgas = dict(zip(hhu_gerit["Einrichtung"], hhu_gerit["Einrichtung"]))

# Zuordnung der Organisationen aus db_hhu zu den GERIT-Organisationen
db_hhu["matched_organisation"] = db_hhu["organisation_mehrere"].apply(
    lambda x: match_organisation(x, gerit_orgas, retriever)
)
# Ergebnisse ausgeben
print(db_hhu)


                                                   titel  \
9970   Koloniale Spuren im Bergischen Land (unter bes...   
10389                   Campaigning - analog und digital   
5485    Praktikum zum Kurs der Zahnärztlichen Radiologie   
5848                  Der Kalte Krieg in Ostmitteleuropa   
9671   FÄLLT AUS: Grünes NRW.  Umweltverschmutzung, B...   
...                                                  ...   
12247  OB/Q2/BP - Blockpraktikum Kinderheilkunde (Sch...   
1092   Modul C: Versuchsplanung und -auswertung: Comp...   
8610   Theater und Medizin - Utopien und Dystopien me...   
738                    Seminar zur Angewandten Statistik   
17403  Aktuelle Methoden der mikrobiologischen Forschung   

                                                     url  \
9970   https://lsf.hhu.de/qisserver/rds?state=verpubl...   
10389  https://lsf.hhu.de/qisserver/rds?state=verpubl...   
5485   https://lsf.hhu.de/qisserver/rds?state=verpubl...   
5848   https://lsf.hhu.de/qisserver/rds

In [ ]:
def match_organisation_optimized(org_series, mapping_dict, retriever):
    """
    Optimierte Funktion zum Abgleichen von Organisationen.
    - Verarbeitet nur einzigartige Werte in org_series mit dem Retriever.
    - Nutzt ein Dictionary zur effizienten Rekodierung der ursprünglichen Daten.
    
    :param org_series: Pandas Series mit den ursprünglichen Organisationen
    :param mapping_dict: Dictionary mit bekannten exakten Zuordnungen
    :param retriever: Retriever zur unscharfen Suche
    :return: Pandas Series mit rekodierten Organisationen
    """
    unique_orgs = org_series.dropna().unique()  # Nur eindeutige, nicht-leere Werte extrahieren
    print(f"Anzahl einzigartiger Organisationen in den Eingabedaten: {len(unique_orgs)}")
    
    recode_dict = {}

    for org in unique_orgs:
        orgs = [o.strip() for o in org.split(';')]  # Aufteilen bei ";", Leerzeichen entfernen
        matched_orgs = []

        for o in orgs:
            # 1. Prüfen, ob eine exakte Übereinstimmung existiert
            if o in mapping_dict:
                matched_orgs.append(mapping_dict[o])
                print(f"Exakte Übereinstimmung gefunden: '{o}' → '{mapping_dict[o]}'")
            else:
                # 2. Falls keine exakte Übereinstimmung, unscharfe Suche über den Retriever
                results = retriever.invoke(o)  # invoke nutzen, da retriever eine Chain ist

                # Sicherstellen, dass ein Ergebnis existiert und den Score-Threshold erfüllt
                if results and results[0].metadata.get("score", 0) >= 0.5:
                    matched_orgs.append(results[0].page_content)
                    print(f"Fuzzy Match gefunden: '{o}' → '{results[0].page_content}' (Score: {results[0].metadata['score']:.2f})")
                else:
                    print(f"Kein passender Match für '{o}' gefunden.")

        recode_dict[org] = "; ".join(matched_orgs) if matched_orgs else None

    # Ursprüngliche Series mit den rekodierten Werten ersetzen
    return org_series.map(recode_dict)


In [ ]:
db_hhu["organisation_matched"] = match_organisation_optimized(db_hhu["organisation_mehrere"], mapping_dict, retriever)


Anzahl einzigartiger Organisationen in den Eingabedaten: 662
Fuzzy Match gefunden: 'Biochemie der Pflanzen' → 'Institut für Biochemie der Pflanzen' (Score: 0.76)
Exakte Übereinstimmung gefunden: 'Department Biologie' → 'Department Biologie'
Fuzzy Match gefunden: 'SFB 1208 - Biochemie' → 'Institut für Biochemie' (Score: 0.71)
Fuzzy Match gefunden: 'Lehrstuhl für Organische Chemie I' → 'Lehrstuhl für Organische Chemie' (Score: 0.93)
Fuzzy Match gefunden: 'Institut für Theoretische Chemie und Computerchemie' → 'Institut für Theoretische Chemie' (Score: 0.91)
Fuzzy Match gefunden: 'Institut für Physikalische Chemie und Elektrochemie' → 'Institut für Physikalische Chemie' (Score: 0.86)
Fuzzy Match gefunden: 'Synthetische Mikrobiologie CEPLAS' → 'Institut für Synthetische Mikrobiologie' (Score: 0.75)
Fuzzy Match gefunden: 'Biochemie der Pflanzen' → 'Institut für Biochemie der Pflanzen' (Score: 0.76)
Fuzzy Match gefunden: 'Pflanzliche Zellbiologie und Biotechnologie' → 'Institut für Pflanzlic